# Deploy Function in WML

In [1]:
import os
import wml_sdk_utils as wml_util

In [2]:
WML_SPACE_ID = '934341dc-0a71-4d86-9c09-a47261359cca'

In [3]:
wml_client = wml_util.get_client(space_id=WML_SPACE_ID)
wml_client.version

'1.0.181'

In [4]:
wml_client.repository.list_functions()

------------------------------------  ------------------------  ------------------------  ------
GUID                                  NAME                      CREATED                   TYPE
d5c27c2d-1924-4454-a89c-9ddbd7e4f775  generic metrics wendy     2022-05-03T20:42:30.002Z  python
5c821656-59cd-42d0-89e7-15583ded5e8f  wml dummy function wendy  2022-05-03T20:42:01.002Z  python
------------------------------------  ------------------------  ------------------------  ------


#### input schema
`{"input_data":[{"values":[<something>]}]}`
- the input has to have a field "input_data"
- the list passed to "values" cannot be empty
- you can have more name-value pairs as you wish, for example:
```
{"input_data": [{"values":[None],"myparam": "abc"}]}
{"input_data": [{"values":[None]}], "myparam": "abc"}
```


#### output schema
`{"predictions":[]}`
- the output has to have a field "predictions"
- you can add your own name-value pairs as you wish similar to the input schema:
```
{"predictions":["myparam":"abc"]}
{"predictions":[],"myparam":"abc"}
```

## Deploy a Dummy Function

In [5]:
path_script = 'wml_deployable_function_dummy.py'
function_name = 'wml dummy function wendy'
deploy_name = 'wml dummy function deploy wendy'

In [6]:
function_asset_id = wml_util.function_store(path_script,wml_client,function_name=function_name)
print(function_asset_id)

Deleted deployment wml dummy function deploy wendy (1bf88c73-f94f-4281-9dba-8f7caeae3d0e) associated with asset wml dummy function wendy (5c821656-59cd-42d0-89e7-15583ded5e8f).
Deleted function asset wml dummy function wendy (5c821656-59cd-42d0-89e7-15583ded5e8f).
Function UID = 2286e733-a493-48f3-b02f-e37790a906ac
deleting tmp/wml_deployable_function_dummy.py.gz...
2286e733-a493-48f3-b02f-e37790a906ac


In [7]:
deployment_id,scoring_url = wml_util.function_deploy(function_asset_id,wml_client,function_deployment_name=deploy_name)
print(deployment_id,scoring_url)



#######################################################################################

Synchronous deployment creation for uid: '2286e733-a493-48f3-b02f-e37790a906ac' started

#######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.
.......
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='bca50a88-f98f-4c9a-ba79-9f5b33693154'
------------------------------------------------------------------------------------------------


bca50a88-f98f-4c9a-ba79-9f5b33693154 https://internal-nginx-svc:12443/ml/v4/deployments/bca50a88-f98f-4c9a-ba79-9f5b33693154/predictions?version=2022-05-03


In [8]:
import requests
import json

headers = {'Authorization':'Bearer '+os.environ['USER_ACCESS_TOKEN'],
           'Content-Type':'application/json'}

payload = {'input_data':[{'values':[None]}],
           'batch_id':1,
           'batch_label':114}

res = requests.post(url=scoring_url,
                    data=json.dumps(payload),
                    headers=headers,
                    verify=False)
res

<Response [200]>

In [9]:
res.content

b'{"predictions": [], "status": "Found.", "true_label": 9, "filename": "camion_s_000148.png"}\n'

## Deploy A Dummy Custom Metric Provider

In [10]:
path_script = 'custom_metrics_dummy.py'
function_name = 'generic metrics wendy'
deploy_name = 'generic metrics deploy wendy'

In [11]:
function_asset_id = wml_util.function_store(path_script,wml_client,function_name=function_name)
print(function_asset_id)

Deleted deployment generic metrics deploy wendy (6e4f7edf-101a-488e-b468-e272bc3857f4) associated with asset generic metrics wendy (d5c27c2d-1924-4454-a89c-9ddbd7e4f775).
Deleted function asset generic metrics wendy (d5c27c2d-1924-4454-a89c-9ddbd7e4f775).
Function UID = 5b21e501-d854-4d15-9b7d-8981554b0c45
deleting tmp/custom_metrics_dummy.py.gz...
5b21e501-d854-4d15-9b7d-8981554b0c45


In [12]:
deployment_id,scoring_url = wml_util.function_deploy(function_asset_id,wml_client,function_deployment_name=deploy_name)
print(deployment_id,scoring_url)



#######################################################################################

Synchronous deployment creation for uid: '5b21e501-d854-4d15-9b7d-8981554b0c45' started

#######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.
...
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='af5ea879-60b8-4f9e-a276-893e1f986a34'
------------------------------------------------------------------------------------------------


af5ea879-60b8-4f9e-a276-893e1f986a34 https://internal-nginx-svc:12443/ml/v4/deployments/af5ea879-60b8-4f9e-a276-893e1f986a34/predictions?version=2022-05-03


#### Sadly you cannot really test this endpoint until you have a real openscale subscription.